<a href="https://colab.research.google.com/github/alheliou/Bias_mitigation/blob/main/TD3_pre_post_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TD 3: Mitigation des biais avec des méthodes de pré-processing et de post-processing

## Objectives:

    1. Use pre-processing approaches, see if they perform well on the use case

    2. Use post-processing approaches, see if they perform well on the use case

    3. Try to combine pre-processing with in-processing and/or with post-processing.


## Installation of the environnement

We highly recommend you to follow these steps, it will allow every student to work in an environment as similar as possible to the one used during testing.

### Colab Settings
  The next two cells of code are too execute only once per colab environment


#### 1. Python env creation

        ```
        ! python -m pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```

#### 2. Download MEPS dataset it can take several minutes

        ```
        ! Rscript /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/generate_data.R
        ! mv h181.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
        ! mv h192.csv /usr/local/lib/python3.12/dist-packages/aif360/data/raw/meps/
        ```

  
### Local Settings

#### 1. Uv installation


        https://docs.astral.sh/uv/getting-started/installation/


        `curl -LsSf https://astral.sh/uv/install.sh | sh`

        Python version 3.12 installation (highly recommended)
        `uv python install 3.12`

#### 2. R installation (needed for data download/pre-processing only of Part 2)

        In the command `Rscript` says 'command not found'

        `sudo apt install r-base-core`

#### 3. Python env creation

        ```
        mkdir TD_bias_mitigation
        cd TD_bias_mitigation
        uv python pin 3.12
        uv init
        uv pip install numpy fairlearn plotly nbformat ipykernel aif360["inFairness"] aif360['AdversarialDebiasing'] causal-learn BlackBoxAuditing cvxpy dice-ml lime shapkit
        ```

#### 4. Download MEPS dataset it can take several minutes

        ```
        cd TD_bias_mitigation/.venv/lib/python3.12/site-packages/aif360/data/raw/meps/
        Rscript generate_data.R
        ```


## 1.Manipulate the dataset

In [ ]:
# imports
import numpy as np
import pandas as pd
import plotly.express as px
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", append=True, category=UserWarning)
# Datasets
from aif360.datasets import MEPSDataset19
from aif360.explainers import MetricTextExplainer

# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from sklearn.metrics import accuracy_score, balanced_accuracy_score


MEPSDataset19_data = MEPSDataset19()
(dataset_orig_panel19_train, dataset_orig_panel19_val, dataset_orig_panel19_test) = (
    MEPSDataset19().split([0.5, 0.8], shuffle=True)
) # 50% train, 30% val, 20% test

In [ ]:
len(dataset_orig_panel19_train.instance_weights), len(
    dataset_orig_panel19_val.instance_weights
), len(dataset_orig_panel19_test.instance_weights)

(7915, 4749, 3166)

In [ ]:
instance_weights = MEPSDataset19_data.instance_weights
instance_weights

array([21854.981705, 18169.604822, 17191.832515, ...,  3896.116219,
        4883.851005,  6630.588948], shape=(15830,))

In [ ]:
f"Taille du dataset {len(instance_weights)}, poids total du dataset {instance_weights.sum()}."

'Taille du dataset 15830, poids total du dataset 141367240.546316.'

### Conversion to a DataFrame

We have seen that the sum of the weights is significant, nearly 115 million, so we cannot reasonably duplicate each row as many times as its weight.

We will store the weighting and take it into account later in our analysis.

In [ ]:
def get_df(MepsDataset):
    data = MepsDataset.convert_to_dataframe()
    # data_train est un tuple, avec le data_frame et un dictionnaire avec toutes les infos (poids, attributs sensibles etc)
    df = data[0]
    df["WEIGHT"] = data[1]["instance_weights"]
    return df


df = get_df(MEPSDataset19_data)

Method that prints the fairness metrics of a dataset

In [ ]:
# Code to compute fairness metrics using aif360

from aif360.sklearn.metrics import *
from sklearn.metrics import  balanced_accuracy_score


# This method takes lists
def get_metrics(
    y_true, # list or np.array of truth values
    y_pred=None,  # list or np.array of predictions
    prot_attr=None, # list or np.array of protected/sensitive attribute values
    priv_group=1, # value taken by the privileged group
    pos_label=1, # value taken by the positive truth/prediction
    sample_weight=None # list or np.array of weights value,
):
    group_metrics = {}
    group_metrics["base_rate_truth"] = base_rate(
        y_true=y_true, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["statistical_parity_difference"] = statistical_parity_difference(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    group_metrics["disparate_impact_ratio"] = disparate_impact_ratio(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
    )
    if not y_pred is None:
        group_metrics["base_rate_preds"] = base_rate(
        y_true=y_pred, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["equal_opportunity_difference"] = equal_opportunity_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["average_odds_difference"] = average_odds_difference(
            y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, priv_group=priv_group, pos_label=pos_label, sample_weight=sample_weight
        )
        if len(set(y_pred))>1:
            group_metrics["conditional_demographic_disparity"] = conditional_demographic_disparity(
                y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
            )
        else:
            group_metrics["conditional_demographic_disparity"] =None
        group_metrics["smoothed_edf"] = smoothed_edf(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["df_bias_amplification"] = df_bias_amplification(
        y_true=y_true, y_pred=y_pred, prot_attr=prot_attr, pos_label=pos_label, sample_weight=sample_weight
        )
        group_metrics["balanced_accuracy_score"] = balanced_accuracy_score(
        y_true=y_true, y_pred=y_pred, sample_weight=sample_weight
        )
    return group_metrics

In [ ]:
get_metrics(
        y_true= df.UTILIZATION,
        y_pred= None,
        prot_attr= df.RACE,
        priv_group= 1,
        pos_label= 0,
        sample_weight= df.WEIGHT,
)

{'base_rate_truth': np.float64(0.7849286063696154),
 'statistical_parity_difference': np.float64(0.1350744772647814),
 'disparate_impact_ratio': 1.1848351529675123}

## Part 2. Apply AIF360 pre-processing methods

In [ ]:
sens_ind = 0
sens_attr = dataset_orig_panel19_train.protected_attribute_names[sens_ind]
unprivileged_groups = [
    {sens_attr: v}
    for v in dataset_orig_panel19_train.unprivileged_protected_attributes[sens_ind]
]
privileged_groups = [
    {sens_attr: v}
    for v in dataset_orig_panel19_train.privileged_protected_attributes[sens_ind]
]
sens_attr, unprivileged_groups, privileged_groups

('RACE', [{'RACE': np.float64(0.0)}], [{'RACE': np.float64(1.0)}])

### 2.1.1 Quesiton: Train a logistic regression to predict the 'UTILIZATION'


### 2.1.2 Question : compute the accuracy and its balanced accuracy (balanced per class), with and without the instance weights

### 2.2 Question: Bias metrics

Compute the metrics on the validation dataset, does the model amplify the bias ?

You can compare the metrics using the truth as the prediction, or random predictions


### 2.2 Reweighing
#### 2.2.1. Question : Find in the [API](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.Reweighing.html) which objects/functions should be used for reweighting and apply them to the datasets

#### 2.2.2. Question: Train a logistic regression of the reweighted training dataset. Then compute the accuracy, balanced accuracy and fairness metrics on the validation dataset

As seens in the lesson, Reweighting change only the instance weights, the features and label remain the same

### 2.3. Disparate Impact Remover
#### 2.3.1. Question : Find in the [API](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.DisparateImpactRemover.html) which objects/functions should be used for disparate impact remover and apply them to the datasets


#### 2.3.2. Question: Train a logistic regression on the transform dataset without the sensitive attribute and compute the fairness metrics.

### 2.4. Question: Train a Latent Fair Representation [API](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.preprocessing.LFR.html)


## Part 3 Post processing

### 3.1 Post-processing [Reject Option Classification](https://aif360.readthedocs.io/en/latest/modules/generated/aif360.algorithms.postprocessing.RejectOptionClassification.html)

#### 3.1.1 Reuse the first Logistic Regression learn to find the best threshold that maximises its balanced accuracy on the validation dataset

#### 3.1.2 Use the RejectOptionClassification  on the validation dataset with the logistic regression predictions. To improve the fairness metrics

#### 3.1.3 Do the same while starting from the Logistic Regression learned on the Reweighted dataset

#### 3.2 Use the Calibrated Equalised Odds  on the validation dataset with the logistic regression predictions. To improve the fairness metrics

## Part 4: Combine pre-processing and in-processing approaches

Combine the ReWeighing with the Prejudice Remover (has seen in TD2, 14th of october)

## Part Optional : Study the impact of Reweighing on different models

Does it impact Decision Tree, why not ?

from sklearn import tree
DTclf = tree.DecisionTreeClassifier()